In [137]:
import os
import sys

sys.path.insert(0, '/kaggle/input/aoa-utils')

from subprocess_runner import run_command
from file_operations import (
    parse_file,
    sanitize_text_for_api,
    save_embeddings,
    load_embeddings
)

from embedding_operations import (
    get_chunks_embeddings,
    get_prompt_embedding,
    find_most_similar,
    query_model_with_context,
    query_model_without_context
)
os.chdir('/kaggle/working/')
print(os.getcwd())

/kaggle/working


In [2]:
run_command("curl -fsSL https://ollama.com/install.sh | sh")
os.system("/usr/local/bin/ollama serve &")

/usr/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
/usr/lib/python3.10/subprocess.py:966: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


0

In [3]:
run_command("ollama pull mistral")
run_command("ollama pull all-minilm:l12-v2")
run_command("curl http://127.0.0.1:11434/api/chat -d '{\"model\": \"mistral\", \"stream\": false, \"messages\": [{ \"role\": \"user\", \"content\": \"Write a sentence as J. D. Salinger would write it\" }]}'")

'{"model":"mistral","created_at":"2025-01-21T12:41:38.294839441Z","message":{"role":"assistant","content":" In the still, quiet afternoon, the sun seemed to shimmer and quiver through the leaves of the weeping willow, and Holden Caulfield, standing on the distant hillside, felt the weight of the world pressing down upon his shoulders like a heavy, unyielding burden."},"done_reason":"stop","done":true,"total_duration":3751807442,"load_duration":2234828991,"prompt_eval_count":18,"prompt_eval_duration":335000000,"eval_count":62,"eval_duration":1179000000}'

In [115]:
filename = "../input/functional-analysis/lecture_notes_functional_analysis.txt"

paragraphs = parse_file(filename)
processed_paragraphs = sanitize_text_for_api(paragraphs)

In [118]:
embeddings = get_chunks_embeddings(filename, "mistral", processed_paragraphs)
print("Embeddings generated successfully!")

Processing chunks: 100%|██████████| 296/296 [02:29<00:00,  1.98chunk/s]


Embeddings generated successfully!


In [181]:
SYSTEM_PROMPT = """You are a helpful teaching assistant that answers only in russian,
    on functional analisys questions based on snippets of text provided in context.
    Answer only using the context provided, being as concise as possible. Give
    straight formal answers on discrete analysis questions as the answer was presented.
    Here are excerpts from the notes, some, i.e. not all of which you may find useful for yourself.
    Don't be afraid to use them:
"""

prompt = "Что такое оператор Вольтерра"
prompt_embedding = get_prompt_embedding(prompt, modelname="mistral")
most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:8]

In [183]:
response = query_model_with_context(
    modelname="mistral",
    system_prompt=SYSTEM_PROMPT,
    prompt=prompt,
    most_similar_chunks=most_similar_chunks,
    paragraphs=processed_paragraphs,
)

if response is not None:
    print("Model response:")
    print(response['content'])
else:
    print("Failed to get a response from the model.")

Model response:
 Оператор Вольтерра - это линейный непрерывный опертор из пространства $E1 = C(0,1)$ в себя же, заданный формулой $(Af)(x) = \int\limits_0^x f(t)dt$. Таким образом, если $f \in E1$, то $Af$ будет функцией из $E2 = C(0,1)$, а именно интегралом $\int\limits_0^x f(t)dt$.


In [184]:
response = query_model_without_context(
    modelname="mistral",
    system_prompt=SYSTEM_PROMPT,
    prompt=prompt
)

if response is not None:
    print("Model response:")
    print(response['content'])
else:
    print("Failed to get a response from the model.")

Model response:
 Оператор Вольтерра - это линейный дифференциальный оператор второго порядка, используемый в теории уравнений дифференциальных уравнений. Он определяется как следующий:

    L[f] = d²f/dt² - df/dt - k^2f

где f(t) - функция, k - константа (коэффициент Лапласа). Данный оператор определяет вид решения дифференциального уравнения 2-го порядка с конечными производными по старшим точкам. Он назван в честь французского математика Жозефа Лювилля (Joseph Liouville), который изучал его приложения к физике, а также итальянского математика Оресте Вильгельма Вольтерра, который ранее изучал схожие операторы.
